In [ ]:
%load_ext autoreload
%autoreload 2

### Import classes and functions

In [ ]:
from tactics import *
from genes import *

### Loading data

In [ ]:
human = dict(name="human",
             sequence_path="data/human_proteins.fasta",
             embedding_path="data/uniprot_human_emb.pkl",
             counts_path="data/human.h5ad",
             column="Cross_species")

mouse = dict(name="mouse",
             sequence_path="data/mouse_proteins.fasta",
             embedding_path="data/uniprot_mouse_emb.pkl",
             counts_path="data/mouse.h5ad",
             column="Cross_species")

### Creating ProtBERT embeddings

In [ ]:
embed_proteins(human)
embed_proteins(mouse)

### Calculating initial gene matches

In [ ]:
match_genes(human, mouse, "data/gene_matches.pkl")

### Creating and training the model

In [ ]:
model = TACTiCS(human, mouse, "data/gene_matches.pkl")
model.train()
model.transfer()

### Plotting results

In [ ]:
model.plot_matches()
model.plot_embeddings()

### Saving model and preprocessed data

In [ ]:
model.save("data/human_mouse")

### Loading model

In [ ]:
model = TACTiCS(folder="data/human_mouse")